# 0. Imports and Data Load

In [2]:
import pandas as pd
import classifiers
import importlib
from igraph import Graph
import igraph as ig
import numpy as np
from ast import literal_eval
import concurrent.futures
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
reactome_raw = pd.read_csv("../../data/processed/metrics/process_raw.csv", sep=',', header=0, index_col=0)
reactome_genepanda = pd.read_csv("../../data/processed/metrics/process_genepanda.csv", sep=',', header=0, index_col=0)
reactome_maxlink = pd.read_csv("../../data/processed/metrics/process_maxlink.csv", sep=',', header=0, index_col=0)

In [4]:
reactome_hypergeometric = pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness = pd.read_csv("../../data/processed/metrics/process_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness = pd.read_csv("../../data/processed/metrics/process_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr = pd.read_csv("../../data/processed/metrics/process_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness= pd.read_csv("../../data/processed/metrics/process_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

reactome_hypergeometric_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

reactome_hypergeometric_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

In [4]:
reactome_hypergeometric = pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0)

graph = Graph.Read_GML("../../data/processed/graph_apid_huri")
reactome_proteins_indexes_df = pd.read_csv("../../data/processed/reactome_proteins_indexes_apid_huri.csv", sep=',', header=0)
reactome_neglogp_ar = np.array(reactome_hypergeometric)

reactome_proteins_indexes_df['protein_index'] = reactome_proteins_indexes_df['protein_index'].apply(literal_eval)
reactome_proteins_indexes_df['proteins_ids'] = reactome_proteins_indexes_df['proteins_ids'].apply(literal_eval)

In [ ]:
reactome_labels = np.zeros(reactome_hypergeometric.shape)
column = 0
for indexes in reactome_proteins_indexes_df['protein_index'].values:
    for pos in indexes:
        reactome_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/reactome_labels_apid_huri.csv", reactome_labels, delimiter=",")

In [5]:
reactome_labels_df = pd.read_csv("../../data/processed/reactome_labels_apid_huri.csv", sep=',', names=reactome_proteins_indexes_df['process'].values)
reactome_labels_df['protein_id'] = graph.vs['name']
reactome_labels_df.set_index('protein_id', inplace=True)

In [5]:
protein80_ppis = np.load("../../data/processed/ppis/ppis_red_protein80_apid_huri.npy", allow_pickle=True)
protein80_raw_list = []
protein_label_list = []
for i in range(1,11):
    protein80_ppis_df = pd.DataFrame(protein80_ppis[i-1,:,0:2]).replace(0, np.nan).dropna()
    proteins_protein80 = set(list(set(protein80_ppis_df[0].values))+list(set(protein80_ppis_df[1].values)))
    reactome_raw_protein80 = pd.read_csv("../../data/processed/metrics/reactome_protein80_"+str(i)+".csv")
    reactome_raw_protein80.columns = reactome_labels_df.columns
    reactome_raw_protein80.index = reactome_labels_df[reactome_labels_df.index.isin(proteins_protein80)].index
    protein80_raw_list.append(reactome_raw_protein80)
    protein_label_list.append(reactome_labels_df[reactome_labels_df.index.isin(proteins_protein80)])

In [7]:
ppi80_hyper_list = []
ppi80_closeness_list = []
ppi80_betweenness_list = []
ppi80_fraction_betweenness_list = []
ppi80_rwr_list = []
label_list = []

for i, df in reactome_hypergeometric_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_hyper_list.append(df)
    label_list.append(reactome_labels_df[reactome_labels_df.index.isin(df.index)])

for i, df in reactome_closeness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_closeness_list.append(df)
    
for i, df in reactome_betweenness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_betweenness_list.append(df)
    
for i, df in reactome_rwr_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_rwr_list.append(df)
    
for i, df in reactome_fraction_betweenness_80.groupby(level=0):
    df.set_index('index', inplace=True)
    ppi80_fraction_betweenness_list.append(df)
    
protein80_hyper_list = []
protein80_closeness_list = []
protein80_betweenness_list = []
protein80_rwr_list = []
protein80_fraction_betweenness_list = []
protein_label_list = []

for i, df in reactome_hypergeometric_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_hyper_list.append(df)
    protein_label_list.append(reactome_labels_df[reactome_labels_df.index.isin(df.index)])
    
for i, df in reactome_closeness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_closeness_list.append(df)
    
for i, df in reactome_betweenness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_betweenness_list.append(df)
    
for i, df in reactome_rwr_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_rwr_list.append(df)
    
for i, df in reactome_fraction_betweenness_protein80.groupby(level=0):
    df.set_index('index', inplace=True)
    protein80_fraction_betweenness_list.append(df)

In [6]:
reactome_raw_test_indices = pd.read_csv("../../data/processed/fs/process_raw_test.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_genepanda_test_indices = pd.read_csv("../../data/processed/fs/reactome_genepanda_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_maxlink_test_indices = pd.read_csv("../../data/processed/fs/reactome_maxlink_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1

In [ ]:
reactome_hypergeometric_test_indices = pd.read_csv("../../data/processed/fs/reactome_hyper_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_closeness_test_indices = pd.read_csv("../../data/processed/fs/reactome_closeness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_betweenness_test_indices = pd.read_csv("../../data/processed/fs/reactome_betweenness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_rwr_test_indices = pd.read_csv("../../data/processed/fs/reactome_rwr_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
reactome_fraction_betweenness_test_indices = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_test_apid_huri.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1

reactome_hypergeometric_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_closeness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_rwr_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_fraction_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1

reactome_hypergeometric_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_closeness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_rwr_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_fraction_betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1

# 1. Threshold Classifier - Complete Network

In [ ]:
importlib.reload(classifiers)
raw_simple_clf = classifiers.threshold_classifier(reactome_raw, reactome_labels_df, reactome_raw_test_indices, op_metric='fmeasure')
raw_simple_clf.to_csv('../../models/threshold_classifier/process/process_raw_simple.csv', index=False)

In [8]:
importlib.reload(classifiers)
genepanda_simple_clf = classifiers.threshold_classifier(reactome_genepanda, reactome_labels_df, reactome_genepanda_test_indices, op_metric='fmeasure')
genepanda_simple_clf.to_csv('../../models/threshold_classifier/process/process_genepanda_simple.csv', index=False)

  0%|          | 0/429 [00:00<?, ?it/s]

In [7]:
importlib.reload(classifiers)
maxlink_simple_clf = classifiers.threshold_classifier(reactome_maxlink, reactome_labels_df, reactome_maxlink_test_indices, op_metric='fmeasure')
maxlink_simple_clf.to_csv('../../models/threshold_classifier/process/process_maxlink_simple.csv', index=False)

  0%|          | 0/429 [00:00<?, ?it/s]

In [ ]:
importlib.reload(classifiers)
hypergeometric_simple_clf = classifiers.threshold_classifier(reactome_hypergeometric, reactome_labels_df, reactome_hypergeometric_test_indices, op_metric='fmeasure')
hypergeometric_simple_clf.to_csv('../../models/threshold_classifier/process/process_hypergeometric_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
closeness_simple_clf = classifiers.threshold_classifier(reactome_closeness, reactome_labels_df, reactome_closeness_test_indices, op_metric='fmeasure')
closeness_simple_clf.to_csv('../../models/threshold_classifier/process/process_closeness_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
betweenness_simple_clf = classifiers.threshold_classifier(reactome_betweenness, reactome_labels_df, reactome_betweenness_test_indices, op_metric='fmeasure')
betweenness_simple_clf.to_csv('../../models/threshold_classifier/process/process_betweenness_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
rwr_simple_clf = classifiers.threshold_classifier(reactome_rwr, reactome_labels_df, reactome_rwr_test_indices, op_metric='fmeasure')
rwr_simple_clf.to_csv('../../models/threshold_classifier/process/process_rwr_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
fraction_betweenness_simple_clf = classifiers.threshold_classifier(reactome_fraction_betweenness, reactome_labels_df, reactome_betweenness_test_indices, op_metric='fmeasure')
fraction_betweenness_simple_clf.to_csv('../../models/threshold_classifier/process/process_fraction_betweenness_simple.csv', index=False)

# 2. Threshold Classifier - Reduced Networks

## 2.1. PPI 80% Reduction

In [ ]:
importlib.reload(classifiers)
hyper80_simple_clf = classifiers.reduced_classifier_threshold(ppi80_hyper_list, reactome_hypergeometric_80_test_index, label_list)
hyper80_simple_clf.to_csv('../../models/threshold_classifier/process/process_hyper80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
rwr80_simple_clf = classifiers.reduced_classifier_threshold(ppi80_rwr_list, reactome_rwr_80_test_index, label_list)
rwr80_simple_clf.to_csv('../../models/threshold_classifier/process/process_rwr80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
closeness80_simple_clf = classifiers.reduced_classifier_threshold(ppi80_closeness_list, reactome_closeness_80_test_index, label_list)
closeness80_simple_clf.to_csv('../../models/threshold_classifier/process/process_closeness80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
betweenness80_simple_clf = classifiers.reduced_classifier_threshold(ppi80_betweenness_list, reactome_betweenness_80_test_index, label_list)
betweenness80_simple_clf.to_csv('../../models/threshold_classifier/process/process_betweenness80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
fraction_betweenness80_simple_clf = classifiers.reduced_classifier_threshold(ppi80_fraction_betweenness_list, reactome_fraction_betweenness_80_test_index, label_list)
fraction_betweenness80_simple_clf.to_csv('../../models/threshold_classifier/process/process_fraction_betweenness80_simple.csv', index=False)

## 2.2. Protein 80% Reduction

In [16]:
importlib.reload(classifiers)
raw_protein80_simple_clf = classifiers.reduced_classifier_threshold(protein80_raw_list, np.moveaxis(reactome_raw_protein80_test_index, 1, 0), protein_label_list)
raw_protein80_simple_clf.to_csv('../../models/threshold_classifier/process/process_raw_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

In [ ]:
importlib.reload(classifiers)
hyper_protein80_simple_clf = classifiers.reduced_classifier_threshold(protein80_hyper_list, reactome_hypergeometric_protein80_test_index, protein_label_list)
hyper_protein80_simple_clf.to_csv('../../models/threshold_classifier/process/process_hyper_protein80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
rwr_protein80_simple_clf = classifiers.reduced_classifier_threshold(protein80_rwr_list, reactome_rwr_protein80_test_index, protein_label_list)
rwr_protein80_simple_clf.to_csv('../../models/threshold_classifier/process/process_rwr_protein80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
closeness_protein80_simple_clf = classifiers.reduced_classifier_threshold(protein80_closeness_list, reactome_closeness_protein80_test_index, protein_label_list)
closeness_protein80_simple_clf.to_csv('../../models/threshold_classifier/process/process_closeness_protein80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
betweenness_protein80_simple_clf = classifiers.reduced_classifier_threshold(protein80_betweenness_list, reactome_betweenness_protein80_test_index, protein_label_list)
betweenness_protein80_simple_clf.to_csv('../../models/threshold_classifier/process/process_betweenness_protein80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
fraction_betweenness_protein80_simple_clf = classifiers.reduced_classifier_threshold(protein80_fraction_betweenness_list, reactome_fraction_betweenness_protein80_test_index, protein_label_list)
fraction_betweenness_protein80_simple_clf.to_csv('../../models/threshold_classifier/process/process_fraction_betweenness_protein80_simple.csv', index=False)